In [2]:
import tensorflow as tf
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Zad.
Dostosuj hiperparametry za pomocą sprawdzania krzyżowego i zobacz, jaką precyzję możesz osiągnąć.


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

X_train = mnist.train.images
y_train = mnist.train.labels

X_valid = mnist.validation.images
y_valid = mnist.validation.labels

X_test = mnist.test.images
y_test = mnist.test.labels

X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]

X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]

X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

he_init = tf.variance_scaling_initializer()

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
        
        
    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver
        
    def close_session(self):
        if self._session:
            self._session.close()

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                for batch_index in range(n_inputs // self.batch_size):
                    print("\r{}%".format(100 * batch_index //  (n_inputs // self.batch_size) ), end="")

                    shuffle_index = np.random.permutation(self.batch_size)
                    X_batch, y_batch = X[shuffle_index], y[shuffle_index]

                    sess.run(self._training_op, feed_dict={self._X: X_batch, self._y: y_batch})
                acc_train = self._accuracy.eval(feed_dict={self._X: X_batch, self._y: y_batch})
                acc_test = self._accuracy.eval(feed_dict={self._X: X_test1, self._y: y_test1})
                print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test) 
#             for epoch in range(n_epochs):
#                 rnd_idx = np.random.permutation(len(X))
#                 for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
#                     X_batch, y_batch = X[rnd_indices], y[rnd_indices]
#                     feed_dict = {self._X: X_batch, self._y: y_batch}
#                     if self._training is not None:
#                         feed_dict[self._training] = True
#                     sess.run(self._training_op, feed_dict=feed_dict)
#                     if extra_update_ops:
#                         sess.run(extra_update_ops, feed_dict=feed_dict)
#                 if X_valid is not None and y_valid is not None:
#                     loss_val, acc_val = sess.run([self._loss, self._accuracy],
#                                                  feed_dict={self._X: X_valid,
#                                                             self._y: y_valid})
#                     if loss_val < best_loss:
#                         best_params = self._get_model_params()
#                         best_loss = loss_val
#                         checks_without_progress = 0
#                     else:
#                         checks_without_progress += 1
#                     print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
#                         epoch, loss_val, best_loss, acc_val * 100))
#                     if checks_without_progress > max_checks_without_progress:
#                         print("Early stopping!")
#                         break
#                 else:
#                     loss_train, acc_train = sess.run([self._loss, self._accuracy],
#                                                      feed_dict={self._X: X_batch,
#                                                                 self._y: y_batch})
#                     print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
#                         epoch, loss_train, acc_train * 100))
#             # If we used early stopping then rollback to the best model found
#             if best_params:
#                 self._restore_model_params(best_params)
#             return self
        
        
    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)        

In [21]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=10, X_valid=X_valid1, y_valid=y_valid1)

97%  0 Train accuracy: 1.0 Test accuracy: 0.5806577
97%  1 Train accuracy: 1.0 Test accuracy: 0.58999807
97%  2 Train accuracy: 1.0 Test accuracy: 0.5913602
97%  3 Train accuracy: 1.0 Test accuracy: 0.5913602
97%  4 Train accuracy: 1.0 Test accuracy: 0.5921385
97%  5 Train accuracy: 1.0 Test accuracy: 0.591944
97%  6 Train accuracy: 1.0 Test accuracy: 0.591944
97%  7 Train accuracy: 1.0 Test accuracy: 0.5921385
97%  8 Train accuracy: 1.0 Test accuracy: 0.59233314
97%  9 Train accuracy: 1.0 Test accuracy: 0.59233314


In [22]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.5923331387429461

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    # you could also try exploring different numbers of hidden layers, different optimizers, etc.
    #"n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    #"optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

# If you have Scikit-Learn 0.18 or earlier, you should upgrade, or use the fit_params argument:
# fit_params={"X_valid": X_valid1, "y_valid": y_valid1, "n_epochs": 1000}
# rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
#                                 fit_params=fit_params, random_state=42, verbose=2)
# rnd_search.fit(X_train1, y_train1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x000000B013E717B8> 
85%  0 Train accuracy: 0.99 Test accuracy: 0.5539988
85%  1 Train accuracy: 1.0 Test accuracy: 0.58007395
85%  2 Train accuracy: 1.0 Test accuracy: 0.582409
85%  3 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  4 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  5 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  6 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  7 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  8 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  9 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  10 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  11 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  12 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  13 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  14 Train accuracy: 1.0 Test accuracy: 0.58163065
85%  15 Train accuracy: 1.0 Test acc

85%  157 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  158 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  159 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  160 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  161 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  162 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  163 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  164 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  165 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  166 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  167 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  168 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  169 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  170 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  171 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  172 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  173 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  174 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  175 T

85%  311 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  312 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  313 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  314 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  315 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  316 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  317 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  318 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  319 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  320 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  321 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  322 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  323 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  324 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  325 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  326 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  327 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  328 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  329 T

85%  467 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  468 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  469 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  470 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  471 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  472 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  473 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  474 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  475 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  476 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  477 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  478 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  479 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  480 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  481 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  482 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  483 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  484 Train accuracy: 1.0 Test accuracy: 0.58182526
85%  485 T

85%  627 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  628 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  629 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  630 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  631 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  632 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  633 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  634 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  635 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  636 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  637 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  638 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  639 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  640 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  641 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  642 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  643 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  644 Train accuracy: 1.0 Test accuracy: 0.58201987
85%  645 T

...

93%  548 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  549 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  550 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  551 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  552 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  553 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  554 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  555 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  556 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  557 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  558 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  559 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  560 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  561 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  562 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  563 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  564 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  565 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  566 T

93%  697 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  698 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  699 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  700 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  701 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  702 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  703 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  704 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  705 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  706 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  707 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  708 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  709 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  710 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  711 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  712 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  713 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  714 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  715 T

93%  848 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  849 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  850 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  851 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  852 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  853 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  854 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  855 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  856 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  857 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  858 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  859 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  860 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  861 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  862 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  863 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  864 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  865 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  866 T

[CV]  n_neurons=90, learning_rate=0.05, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000000B0227E8510>, total= 1.0min
[CV] n_neurons=90, learning_rate=0.05, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x000000B0227E8510> 
93%  0 Train accuracy: 0.8 Test accuracy: 0.60400856
93%  1 Train accuracy: 0.96 Test accuracy: 0.8028799
93%  2 Train accuracy: 0.94 Test accuracy: 0.78283715
93%  3 Train accuracy: 0.98 Test accuracy: 0.80132324
93%  4 Train accuracy: 0.86 Test accuracy: 0.6744503
93%  5 Train accuracy: 0.7 Test accuracy: 0.5129403
93%  6 Train accuracy: 0.98 Test accuracy: 0.7511189
93%  7 Train accuracy: 0.94 Test accuracy: 0.80638254
93%  8 Train accuracy: 0.78 Test accuracy: 0.62813777
93%  9 Train accuracy: 1.0 Test accuracy: 0.79840434
93%  10 Train accuracy: 0.88 Test accuracy: 0.53823704
93%  11 Train accuracy: 0.96 Test accuracy: 0.67873126
93%  12 Train accuracy: 0.98 Test accuracy: 0.75442696
9

93%  147 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  148 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  149 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  150 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  151 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  152 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  153 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  154 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  155 Train accuracy: 1.0 Test accuracy: 0.76999414
93%  156 Train accuracy: 1.0 Test accuracy: 0.76999414
93%

In [ ]:
rnd_search_bn.best_params_